# Clustering Crypto

In [69]:
# Initial imports
import requests
import pandas as pd
import matplotlib.pyplot as plt
import hvplot.pandas
import plotly.express as px
from pathlib import Path
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans

### Fetching Cryptocurrency Data

In [70]:
# Use the following endpoint to fetch json data
#url = "https://min-api.cryptocompare.com/data/all/coinlist"

In [71]:
# Create a DataFrame 
# HINT: You will need to use the 'Data' key from the json response, then transpose the DataFrame

In [72]:
# Alternatively, use the provided csv file:
file_path = Path("Resources/crypto_data.csv")

# Create a DataFrame
crypto_df = pd.read_csv(file_path, index_col=0)
crypto_df.tail(10)

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
GAP,Gapcoin,Scrypt,True,PoW/PoS,1.493105e+07,250000000
SERO,Super Zero,Ethash,True,PoW,NaN,1000000000
UOS,UOS,SHA-256,True,DPoI,NaN,1000000000
BDX,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,True,PoW,7.296538e+06,21000000
XBC,BitcoinPlus,Scrypt,True,PoS,1.283270e+05,1000000
DVTC,DivotyCoin,Scrypt,False,PoW/PoS,2.149121e+07,100000000
GIOT,Giotto Coin,Scrypt,False,PoW/PoS,NaN,233100000
OPSC,OpenSourceCoin,SHA-256,False,PoW/PoS,NaN,21000000
PUNK,SteamPunk,PoS,False,PoS,NaN,40000000


### Data Preprocessing

In [73]:
# Keep only necessary columns:
# 'CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply'
trading = crypto_df[['CoinName','Algorithm','IsTrading','ProofType','TotalCoinsMined','TotalCoinSupply']]
trading.tail(10)

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
GAP,Gapcoin,Scrypt,True,PoW/PoS,1.493105e+07,250000000
SERO,Super Zero,Ethash,True,PoW,NaN,1000000000
UOS,UOS,SHA-256,True,DPoI,NaN,1000000000
BDX,Beldex,CryptoNight,True,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,True,PoW,7.296538e+06,21000000
XBC,BitcoinPlus,Scrypt,True,PoS,1.283270e+05,1000000
DVTC,DivotyCoin,Scrypt,False,PoW/PoS,2.149121e+07,100000000
GIOT,Giotto Coin,Scrypt,False,PoW/PoS,NaN,233100000
OPSC,OpenSourceCoin,SHA-256,False,PoW/PoS,NaN,21000000
PUNK,SteamPunk,PoS,False,PoS,NaN,40000000


In [74]:
# Keep only cryptocurrencies that are trading
trading=crypto_df[crypto_df.IsTrading != False]
trading.head(10)

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,True,PoW/PoS,2.927942e+10,314159265359
2015,2015 coin,X11,True,PoW/PoS,NaN,0
BTC,Bitcoin,SHA-256,True,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,True,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,True,PoW,6.303924e+07,84000000


In [75]:
# Keep only cryptocurrencies with a working algorithm
for column in trading.columns:
    print(f"Column {column} has {trading[column].isnull().sum()} null values")

Column CoinName has 0 null values
Column Algorithm has 0 null values
Column IsTrading has 0 null values
Column ProofType has 0 null values
Column TotalCoinsMined has 459 null values
Column TotalCoinSupply has 0 null values


In [76]:
# Remove the 'IsTrading' column
trading.drop(columns=["IsTrading"], inplace=True)
trading.head(10)

/Users/gregbuiter/opt/anaconda3/envs/pyvizenv/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
2015,2015 coin,X11,PoW/PoS,NaN,0
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,PoW,6.303924e+07,84000000


In [77]:
# Remove rows with at least 1 null value
df = trading.dropna(axis=0) 
for column in df.columns:
    print(f"Column {column} has {df[column].isnull().sum()} null values")

Column CoinName has 0 null values
Column Algorithm has 0 null values
Column ProofType has 0 null values
Column TotalCoinsMined has 0 null values
Column TotalCoinSupply has 0 null values


In [78]:
# Remove rows with cryptocurrencies having no coins mined
df.head(10)

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,PoW,6.303924e+07,84000000
DASH,Dash,X11,PoW/PoS,9.031294e+06,22000000
XMR,Monero,CryptoNight-V7,PoW,1.720114e+07,0
ETC,Ethereum Classic,Ethash,PoW,1.133597e+08,210000000


In [79]:
df2 = df[df.TotalCoinsMined !=0]
df2.head(10)

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
LTC,Litecoin,Scrypt,PoW,6.303924e+07,84000000
DASH,Dash,X11,PoW/PoS,9.031294e+06,22000000
XMR,Monero,CryptoNight-V7,PoW,1.720114e+07,0
ETC,Ethereum Classic,Ethash,PoW,1.133597e+08,210000000
ZEC,ZCash,Equihash,PoW,7.383056e+06,21000000


In [80]:
# Drop rows where there are 'N/A' text values
df2 = df2.dropna() 

In [81]:
# Store the 'CoinName'column in its own DataFrame prior to dropping it from crypto_df 
coin_name = pd.DataFrame(df2["CoinName"],index=crypto_df2.index)
coin_name.head()

,CoinName
42,42 Coin
365,NaN
404,404Coin
611,NaN
1337,EliteCoin


In [82]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm
df2.drop(columns=["CoinName"], inplace=True)
df2.head(10)

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0
LTC,Scrypt,PoW,6.303924e+07,84000000
DASH,X11,PoW/PoS,9.031294e+06,22000000
XMR,CryptoNight-V7,PoW,1.720114e+07,0
ETC,Ethash,PoW,1.133597e+08,210000000
ZEC,Equihash,PoW,7.383056e+06,21000000


In [83]:
# Create dummy variables for text features
dummies_proof = pd.get_dummies(df["Algorithm"])
dummies_algo = pd.get_dummies(df["ProofType"])
df2.drop(columns=["Algorithm","ProofType"], inplace=True)
X = pd.concat([df2, dummies_proof,dummies_algo], axis=1,sort=True).reindex(df2.index)
X.head()

,TotalCoinsMined,TotalCoinSupply,1GB AES Pattern Search,536,Argon2,Argon2d,BLAKE256,Blake,Blake2S,Blake2b,...,PoW/PoS,PoW/PoS,PoW/PoW,PoW/nPoS,Pos,Proof of Authority,Proof of Trust,TPoS,Zero-Knowledge Proof,dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [84]:
# Standardize data
X_scaled = StandardScaler().fit_transform(X)

### Reducing Dimensions Using PCA

In [85]:
# Use PCA to reduce dimensions to 3 principal components
pca = PCA(n_components=3)

X_pca = pca.fit_transform(X_scaled)

In [86]:
# Create a DataFrame with the principal components data
pcs_df = pd.DataFrame(
    data=X_pca, columns=["PCA 1", "PCA 2", "PCA 3"]
)
pcs_df.index=df2.index
pcs_df.head(10)


,PCA 1,PCA 2,PCA 3
42,-0.323757,0.891000,-0.586263
404,-0.307124,0.891261,-0.586736
1337,2.318844,1.764511,-0.634260
BTC,-0.136597,-1.268501,0.157105
ETH,-0.156541,-1.949418,0.336576
LTC,-0.179071,-1.241420,-0.014722
DASH,-0.340782,1.329613,-0.525886
XMR,-0.179453,-2.151018,0.443185
ETC,-0.154987,-1.949498,0.336561
ZEC,-0.123165,-1.794270,0.342324


### Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [87]:
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of k values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)

# Create the Elbow Curve using hvPlot
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

:Curve   [k]   (inertia)

Running K-Means with `k=<your best value for k here>`

In [88]:
# Initialize the K-Means model
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(pcs_df)

# Predict clusters
predictions = model.predict(pcs_df)

# Create a new DataFrame including predicted clusters and cryptocurrencies features
pcs_df["Class"] = model.labels_
pcs_df.head()

df2=df[df.TotalCoinsMined !=0 ]
crypto_df= pd.concat([df2,pcs_df],axis=1,sort=True).reindex(df2.index)
column_names=["Algorithm","ProofType","TotalCoinsMined","TotalCoinSupply","PCA 1","PCA 2","PCA 3","CoinName","Class"]
crypto_df = crypto_df.reindex(columns=column_names)
crypto_df.head(10)

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PCA 1,PCA 2,PCA 3,CoinName,Class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.323757,0.891000,-0.586263,42 Coin,1
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.307124,0.891261,-0.586736,404Coin,1
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.318844,1.764511,-0.634260,EliteCoin,1
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.136597,-1.268501,0.157105,Bitcoin,0
ETH,Ethash,PoW,1.076842e+08,0,-0.156541,-1.949418,0.336576,Ethereum,0
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.179071,-1.241420,-0.014722,Litecoin,0
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.340782,1.329613,-0.525886,Dash,1
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.179453,-2.151018,0.443185,Monero,0
ETC,Ethash,PoW,1.133597e+08,210000000,-0.154987,-1.949498,0.336561,Ethereum Classic,0
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.123165,-1.794270,0.342324,ZCash,0


### Visualizing Results

#### 3D-Scatter with Clusters

In [89]:
# Create a 3D-Scatter with the PCA data and the clusters
fig = px.scatter_3d(
    crypto_df,
    x="PCA 3",
    y="PCA 2",
    z="PCA 1",
    color="Class",
    symbol="Class",
    hover_name="CoinName",
    hover_data=["Algorithm"],
    width=800,
)
fig.update_layout(legend=dict(x=0, y=1))
fig.show()


#### Table of Tradable Cryptocurrencies

In [101]:
# Table with tradable cryptos
crypto_df.hvplot.table(columns=["CoinName", "Algorithm", "ProofType", "TotalCoinSupply", "TotalCoinsMined", "class"], width=400)

DataError: Supplied data does not contain specified dimensions, the following dimensions were not found: ['class']

PandasInterface expects tabular data, for more information on supported datatypes see http://holoviews.org/user_guide/Tabular_Datasets.html

In [95]:
# Print the total number of tradable cryptocurrencies
crypto_df.groupby(['CoinName']).sum()

,TotalCoinsMined,PCA 1,PCA 2,PCA 3,Class
CoinName,,,,,
ClubCoin,1.036219e+08,-0.321361,0.890983,-0.586316,1
1717 Masonic Commemorative Token,1.618033e+06,-0.303431,-0.005914,0.087329,1
1Credit,8.821300e+04,0.470073,-1.277486,-0.020546,0
404Coin,1.055185e+09,-0.307124,0.891261,-0.586736,1
42 Coin,4.199995e+01,-0.323757,0.891000,-0.586263,1
8BIT Coin,1.467841e+06,-0.323739,0.891001,-0.586263,1
AC3,8.031621e+07,-0.192857,-0.802977,0.045624,0
ALAX,1.000000e+09,3.832189,1.532042,-0.301649,1
ARENON,1.926951e+07,-0.342709,1.140741,-0.203611,1


### There are 532 tradable cryptocurrencies.

#### Scatter Plot with Tradable Cryptocurrencies

In [99]:
# Scale data to create the scatter plot
crypto_df.describe()

,TotalCoinsMined,PCA 1,PCA 2,PCA 3,Class
count,5.330000e+02,5.330000e+02,5.330000e+02,5.330000e+02,533.000000
mean,5.319333e+09,1.053983e-16,-1.176878e-16,7.415373e-17,0.560976
std,4.560537e+10,1.655597e+00,1.445694e+00,1.418718e+00,0.543707
min,-5.917978e+09,-8.870085e-01,-2.610738e+00,-1.297453e+00,0.000000
25%,8.305775e+06,-3.235411e-01,-1.241483e+00,-5.258840e-01,0.000000
50%,2.448794e+07,-1.974478e-01,6.955192e-01,-6.034301e-02,1.000000
75%,1.835340e+08,-1.454187e-01,1.329460e+00,1.479614e-01,1.000000
max,9.899887e+11,3.406122e+01,3.778912e+00,2.119585e+01,3.000000


In [100]:
# Plot the scatter with x="TotalCoinsMined" and y="TotalCoinSupply"
crypto_df.hvplot.scatter(
    x="TotalCoinsMined",
   y="TotalCoinSupply",
   hover_cols=["CoinName"],
)


:Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)